In [80]:
#importing libraries
import nltk
import pandas as pd
#from emoticons import EmoticonDetector
import re as regex

import string
import collections
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.wsd import lesk
from nltk.corpus import wordnet
import operator
import numpy as np
from nltk.stem.snowball import SnowballStemmer
import plotly
from plotly import graph_objs
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
import time
import gensim
from gensim import corpora
import csv
import _pickle as cPickle
from nltk.corpus import words, stopwords

In [81]:

def strip_links(text):
    link_regex =  regex.compile('(((http?)|(https?)):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', regex.DOTALL)
    links         = regex.findall(link_regex, text)
    for link in links:
        i=0
        text = text.replace(link[i],'')
        i+=1
    return text


def strip_all_entities(text):
    entity_prefixes = ['@', '#', 'virgin', 'america', 'united', 'delta', 'jetblue', 'southwest']
    for separator in string.punctuation:
        if separator not in entity_prefixes:
            text = text.replace(separator, " ")
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                # words.append(" ")
                words.append(word)
            # print(type(''.join(words)))
    return " ".join(words)


def clean(doc):

    stop = set(stopwords.words('english'))

    exclude = set(string.punctuation)
    lemma = WordNetLemmatizer()
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    # print(stop_free)
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    # print(punc_free)

    normalized = " ".join(lemma.lemmatize(word) for word in stop_free.split())


    return (normalized)



def get_clean_docs(documents):
    doc_complete = [doc for doc in documents]
    nltk.download('stopwords')
    nltk.download('wordnet')
    sp = [strip_links(doc) for doc in doc_complete]
    sp = [strip_all_entities(doc) for doc in sp]

    doc_clean = [clean(doc) for doc in sp]

    doc_clean = [regex.sub(r'(.)\1+', r'\1\1', (doc)).split() for doc in doc_clean]
    stemmer2 = SnowballStemmer("english", ignore_stopwords=False)

    for doc in doc_clean:
        if 'u' in doc:
            doc.remove('u')

    print (doc_clean)
    return  doc_clean

def get_dicts():
    df = pd.read_csv('../data/Tweets.csv')
    doc_complete = [row[1] for row in df[df.columns[10]].iteritems()]
    nltk.download('stopwords')
    nltk.download('wordnet')
    sp = [strip_links(doc) for doc in doc_complete]
    sp = [strip_all_entities(doc) for doc in sp]

    doc_clean = [clean(doc) for doc in sp]

    doc_clean = [regex.sub(r'(.)\1+', r'\1\1', (doc)).split() for doc in doc_clean]
    stemmer2 = SnowballStemmer("english", ignore_stopwords=False)

    for doc in doc_clean:
        if 'u' in doc:
            doc.remove('u')

    return doc_clean

In [82]:
Lda = gensim.models.ldamodel.LdaModel
tweet = input()
ldamodel=Lda.load('../model/lda_10')
dclean=get_dicts()
tweet=[tweet]
clean_tweet= get_clean_docs(tweet)
dictionary = corpora.Dictionary(dclean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in clean_tweet]
topics = ldamodel.show_topics(num_topics=5, num_words=10, log=False, formatted=True)
print(topics)

@VirginiaAirlines your customer service is very bad
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\haris\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\haris\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\haris\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\haris\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[['customer', 'service', 'bad']]
[(0, '0.020*"flight" + 0.017*"thank" + 0.010*"aa" + 0.008*"”" + 0.008*"fleet" + 0.008*"fleek" + 0.007*"get" + 0.006*"weather" + 0.006*"dca" + 0.006*"good"'), (1, '0.049*"flight" + 0.026*"hour" + 0.017*"2" + 0.015*"time" + 0.014*"bag" + 0.013*"plane" + 0.012*"get" + 0.011*"delayed" +

In [83]:
label_list=[max(
        ldamodel.get_document_topics(doc, minimum_probability=None, minimum_phi_value=None, per_word_topics=False),
key=lambda item: item[1])[0] for doc in doc_term_matrix]

print(label_list)

[3]


In [84]:
topic_count={}
for label in label_list:
    if label in topic_count:
        topic_count[label]=topic_count[label] + 1
    else:
        topic_count[label]=1
se = pd.Series(label_list)
print (topic_count)
#df = pd.read_csv('Tweets.csv')
df=pd.DataFrame({'text':tweet})
df['label'] = se
#df.to_csv('modified_tweets5.csv')
df.to_csv('../data/LDAOutput.csv')

{3: 1}


In [85]:
class TwitterData_Initialize():
    processed_Traindata = []
    wordlist = []  
    data_model = None
    data_labels = None
    
    def initialize(self, csv_file, from_cached=None):
        if from_cached is not None:
            self.data_model = pd.read_csv(from_cached)
            return
        self.processed_Traindata = pd.read_csv(csv_file, usecols=[0,1])
        self.wordlist = []
        self.data_model = None
        self.data_labels = None

In [86]:
#Data Pre-processing for Sentiment Analysis
data = TwitterData_Initialize()
data.initialize("../data/LDAOutput.csv")
data.processed_Traindata

,Unnamed: 0,text
0,0,@VirginiaAirlines your customer service is ver...


In [87]:
#Data Preprocessing
nltk.download('words')
word_dictionary = list(set(words.words()))

for alphabet in "bcdefghjklmnopqrstuvwxyzBCDEFGHJKLMNOPQRSTUVWXYZ":
    word_dictionary.remove(alphabet)
class DataPreprocessing:
    def iterate(self):
        for preprocessingMethod in [self.replaceProcessedHashtags,
                                   self.removeUrls,
                                   self.removeUsernames,
                                   self.removeElongatedWords,
                                   self.removeNa,
                                   self.replaceSlangWords,
                                   self.removeSpecialChars,
                                   self.removeNumbers]:
            yield preprocessingMethod
    
    @staticmethod
    def removeByRegex(tweets, regExp):
        tweets.loc[:, "text"].replace(regExp, "", inplace=True)
        return tweets
    
    def removeUrls(self, tweets):
        return DataPreprocessing.removeByRegex(tweets, regex.compile(r"http.?://[^\s]+[\s]?"))
    
    def removeNa(self, tweets):
        return tweets[tweets["text"] != ""]
    
    def removeSpecialChars(self, tweets):
        for remove in map(lambda r: regex.compile(regex.escape(r)), [",", ":", "\"", "=", "&", ";", "%", "$"
                                                                    "@", "%", "^", "*", "(", ")", "{", "}",
                                                                    "[", "]", "|", "/", "\\", ">", "<", "-",
                                                                    "!", "?", ".", "'",
                                                                    "--", "---","#"]):
            tweets.loc[:, "text"].replace(remove, "", inplace=True)
        return tweets
    
    def removeUsernames(self, tweets):
        return DataPreprocessing.removeByRegex(tweets, regex.compile(r"@[^\s]+[\s]?"))
    
    def removeElongatedWords(self, tweets):
        return DataPreprocessing.removeByRegex(tweets, regex.compile(r"(.)\1+', r'\1\1"))
    
    def removeNumbers(self, tweets):
        #print(tweets)
        return DataPreprocessing.removeByRegex(tweets, regex.compile(r"\s?[0-9]+\.?[0-9]*"))
    
    def replaceSlangWords(self, tweets):
        with open('../data/slang.txt') as file:
            slang_map = dict(map(str.strip, line.partition('\t')[::2])
            for line in file if line.strip())
            #print(tweets["text"])
            #print("-----------------------------------------END")
            for index,word in tweets['text'].iteritems():
                #print(index)
                for i in word.split():
                    isUpperCase = i.isupper()
                    i = i.lower()
                    if i in slang_map.keys():
                        word = word.replace(i, slang_map[i])
                        tweets.loc[(index),"text"] = word
                if isUpperCase:
                    i = i.upper()
        #print(tweets.loc[:,"text"])
        return tweets

    # print(split_tweets)
    @staticmethod
    def removeDigitsFromHashtag(tag):
        tag = regex.sub(r"\s?[0-9]+\.?[0-9]*", "", tag)
        return tag

    @staticmethod
    def collect_hashtags_in_tweet(wordList):
        hashtags = []
        for word in wordList:
            index = word.find('#')
            if index != -1:
                if word[index + 1:] != '':
                    hashtags.append(word[index + 1:])
        return hashtags

    @staticmethod
    def split_hashtag_to_words_all_possibilities(hashtag):
        all_possibilities = []

        split_posibility = [hashtag[:i] in word_dictionary for i in reversed(range(len(hashtag) + 1))]
        possible_split_positions = [i for i, x in enumerate(split_posibility) if x == True]

        for split_pos in possible_split_positions:
            split_words = []
            word_1, word_2 = hashtag[:len(hashtag) - split_pos], hashtag[len(hashtag) - split_pos:]

            if word_2 in word_dictionary:
                split_words.append(word_1)
                split_words.append(word_2)
                all_possibilities.append(split_words)

                another_round = DataPreprocessing.split_hashtag_to_words_all_possibilities(word_2)

                if len(another_round) > 0:
                    all_possibilities = all_possibilities + [[a1] + a2 for a1, a2, in
                                                             zip([word_1] * len(another_round), another_round)]
            else:
                another_round = DataPreprocessing.split_hashtag_to_words_all_possibilities(word_2)

                if len(another_round) > 0:
                    all_possibilities = all_possibilities + [[a1] + a2 for a1, a2, in
                                                             zip([word_1] * len(another_round), another_round)]

        return all_possibilities

    @staticmethod
    def process_all_hashtags_in_tweet(hashtags):
        all_words = []
        for tag in hashtags:
            split_hashtag = DataPreprocessing.split_hashtag_to_words_all_possibilities(DataPreprocessing.removeDigitsFromHashtag(tag))
            if split_hashtag:
                all_words = all_words + split_hashtag[0]
            else:
                all_words.append(tag)
        return all_words 
    
    def replaceProcessedHashtags(self, tweets):
            for index,word in tweets['text'].iteritems():
                word=word.split()
                collectHashtags=DataPreprocessing.collect_hashtags_in_tweet(word)
                allHashtags=DataPreprocessing.process_all_hashtags_in_tweet(collectHashtags)
                collectHashtags = ["#" + tag for tag in collectHashtags]
                if allHashtags:
                    word = list(set(word) - set(collectHashtags))
                    word = word + allHashtags
                word = " ".join(word)
                tweets.loc[(index),"text"] = word
                #print(tweets.loc[(index),"text"])
           # print(tweets)
            return tweets

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\haris\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [88]:
#Cleaning the Training Data
class CleanTrainingData(TwitterData_Initialize):
    def __init__(self, previous):
        self.processed_Traindata = previous.processed_Traindata
        #self.processed_Testdata = previous.processed_Testdata
    
    def cleaningData(self, cleaner):
        train = self.processed_Traindata
        #test = self.processed_Testdata
        
        for cleanerMethod in cleaner.iterate():
            train = cleanerMethod(train)
            #test = cleanerMethod(test)
        self.processed_Traindata = train
        #self.processed_Testdata = test

In [89]:
data = CleanTrainingData(data)
data.cleaningData(DataPreprocessing())
data.processed_Traindata.head(5)

,Unnamed: 0,text
0,0,your customer service is very bad


In [90]:
#Tokenizing and Stemming the data
class TokenizationStemming(CleanTrainingData):
    def __init__(self, previous):
        self.processed_Traindata = previous.processed_Traindata
        #self.processed_TestData = previous.processed_TestData
    
    def stem(self, stemmer = nltk.PorterStemmer()):
        def stemJoin(row):
            row["text"] = list(map(lambda str: stemmer.stem(str.lower()), row["text"]))
            return row
    
        self.processed_Traindata = self.processed_Traindata.apply(stemJoin, axis=1)
    
    def tokenize(self, tokenizer = nltk.word_tokenize):
        def tokenizeRow(row):
            row["text"] = tokenizer(row["text"])
            row["tokenizedText"] = [] + row["text"]
            return row
        
        self.processed_Traindata = self.processed_Traindata.apply(tokenizeRow, axis=1)

In [91]:
#Building Wordlist
#Un-filtered version without removing stopwords
words = collections.Counter()
for idx in data.processed_Traindata.index:
    words.update(data.processed_Traindata.loc[idx, "text"])

words.most_common(5)

[(' ', 5), ('r', 4), ('e', 4), ('s', 3), ('y', 2)]

In [92]:
#Removing stopwords
stopwords = nltk.corpus.stopwords.words("english")
whitelist = ["n't", "not"]
for idx, stop_word in enumerate(stopwords):
    if stop_word not in whitelist:
        del words[stop_word]

words.most_common(5)

[(' ', 5), ('r', 4), ('e', 4), ('u', 2), ('c', 2)]

In [93]:
#Generating the final wordlist
class WordList(TokenizationStemming):
    def __init__(self, previous):
        self.processed_Traindata = previous.processed_Traindata
    
    whitelist = ["n't", "not"]
    wordlist = []
    
    def buildWordlist(self, min_occurrences=3, max_occurences=3000, stopwords=nltk.corpus.stopwords.words("english"),
                     whitelist=None):
        self.wordlist = []
        whitelist = self.whitelist if whitelist is None else whitelist
        import os
        if os.path.isfile('../data/wordlist.csv'):
            word_df = pd.read_csv('../data/wordlist.csv', encoding = "ISO-8859-1")
            word_df = word_df[word_df["occurrences"] > min_occurrences]
            self.wordlist = list(word_df.loc[:, "word"])
            return
        words = collections.Counter()
        for idx in self.processed_Traindata.index:
            words.update(self.processed_Traindata.loc[idx, "text"])
        
        for idx, stop_word in enumerate(stopwords):
            if stop_word not in whitelist:
                del words[stop_word]
        
        word_df = pd.DataFrame(data={"word" : [k for k, v in words.most_common() if min_occurrences < v < max_occurences],
                                    "occurrences": [v for k, v in words.most_common() if min_occurrences < v < max_occurences]},
                              columns = ["word", "occurrences"])
        
        word_df.to_csv("../data/wordlist.csv", index_label="idx", encoding = "utf8")
        self.wordlist = [k for k, v in words.most_common() if min_occurrences < v < max_occurences]
    

In [94]:
data = WordList(data)
data.buildWordlist()

In [95]:
#Transforming into Bag-of-Words
class BagOfWords(WordList):
    def __init__(self, previous):
        self.processed_Traindata = previous.processed_Traindata
        self.wordlist = previous.wordlist
    
    def buildDataModel(self):
        columns = list(
            map(lambda w: w + "_bow", self.wordlist))
        labels = []
        rows = []
        
        for idx in self.processed_Traindata.index:
            currentRow = []
            
            tokens = set(self.processed_Traindata.loc[idx, "text"])
            for _, word in enumerate(self.wordlist):
                currentRow.append(1 if word in tokens else 0)
            
            rows.append(currentRow)
        
        self.data_model = pd.DataFrame(rows, columns=columns)
        self.data_labels = pd.Series(labels)
        
        return self.data_model, self.data_labels

In [96]:
data = BagOfWords(data)
bow, labels = data.buildDataModel()
bow.head(5)

,hour_bow,cancel_bow,help_bow,servic_bow,delay_bow,time_bow,custom_bow,bag_bow,call_bow,im_bow,...,gorgeou_bow,woohoo_bow,thousand_bow,understat_bow,furiou_bow,manual_bow,smell_bow,ber_bow,charleston_bow,nrt_bow
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [97]:
#Detecting Emoticons
class EmoticonDetector:
    emoticons = {}
    
    def __init__(self, emoticon_file="../data/emoticons.txt"):
        from pathlib import Path
        content = Path(emoticon_file).read_text()
        positive = True
        for line in content.split("\n"):
            if "positive" in line.lower():
                positive = True
                continue
            elif "negative" in line.lower():
                positive = False
                continue
            
            self.emoticons[line] = positive
    
    def is_positive(self, emoticon):
        if emoticon in self.emoticons:
            return self.emoticons[emoticon]
        return False
    
    def is_emoticon(self, to_check):
        return to_check in self.emoticons

In [98]:
class ExtraFeatures(WordList):
    def __init__(self):
        pass
    
    def build_data_model(self):
        extra_columns = [col for col in self.processed_Traindata.columns if col.startswith("number_of")]
        columns = extra_columns + list(
                map(lambda w: w + "_bow", self.wordlist))
        
        labels = []
        rows = []
        for idx in self.processed_Traindata.index:
            current_row = []
        
            for _,col in enumerate(extra_columns):
                current_row.append(self.processed_Traindata.loc[idx, col])
        
        #adding bag-of-words
            tokens = set(self.processed_Traindata.loc[idx, "text"])
            for _,word in enumerate(self.wordlist):
                current_row.append(1 if word in tokens else 0)
        
            rows.append(current_row)
        
        self.data_model = pd.DataFrame(rows, columns=columns)
        self.data_labels = pd.Series(labels)
        
        return self.data_model, self.data_labels
    
    def add_column(self, column_name, column_content):
        self.processed_Traindata.loc[:, column_name] = pd.Series(column_content, index=self.processed_Traindata.index)

    def build_features(self):
        def count_by_lambda(expression, word_array):
            return len(list(filter(expression, word_array)))
        
        def count_occurences(character, word_array):
            counter = 0
            for j, word in enumerate(word_array):
                for char in word:
                    if char == character:
                        counter += 1
            return counter
        
        def count_interjections(wordArray):
            interjections = []
            interjectionCount = 0
            with open('../data/interjections.txt') as file:
                interjections = file.read().splitlines()
            for word in wordArray:
                if word in interjections:
                    interjectionCount += 1
            return interjectionCount 

        def count_by_regex(regex, plain_text):
            return len(regex.findall(plain_text))
        
        self.add_column("splitted_text", map(lambda txt: txt.split(" "), self.processed_Traindata["text"]))
        
        #Number of uppercase words
        uppercase = list(map(lambda txt: count_by_lambda(lambda word: word == word.upper(), txt),
                                                        self.processed_Traindata["splitted_text"]))
        self.add_column("number_of_uppercase", uppercase)
        
        #number of !
        exclamations = list(map(lambda txt: count_occurences("!", txt),
                               self.processed_Traindata["splitted_text"]))
        self.add_column("number_of_exclamation", exclamations)
        
        #number of ?
        questions = list(map(lambda txt: count_occurences("?", txt),
                               self.processed_Traindata["splitted_text"]))
        self.add_column("number_of_question", questions)
        
        #number of ...
        ellipsis = list(map(lambda txt: count_by_regex(regex.compile(r"\.\s?\.\s?\."), txt),
                           self.processed_Traindata["text"]))
        self.add_column("number_of_ellipsis", ellipsis)
        
        #number of hashtags
        hashtags = list(map(lambda txt: count_occurences("#", txt),
                               self.processed_Traindata["splitted_text"]))
        self.add_column("number_of_hashtags", hashtags)
        
        #number of mentions
        mentions = list(map(lambda txt: count_occurences("@", txt),
                               self.processed_Traindata["splitted_text"]))
        self.add_column("number_of_mentions", mentions)
        
        #number of quotes
        quotes = list(map(lambda plain_text: int(count_occurences("'", [plain_text.strip("'").strip('"')]) / 2 +
                                                 count_occurences('"', [plain_text.strip("'").strip('"')]) / 2),
                          self.processed_Traindata["text"]))
        self.add_column("number_of_quotes", quotes)
        
        #number of urls
        urls = list(map(lambda txt: count_by_regex(regex.compile(r"http.?://[^\s]+[\s]?"), txt),
                             self.processed_Traindata["text"]))
        self.add_column("number_of_urls", urls)
        
        #number of positive emoticons
        ed = EmoticonDetector()
        positive_emo = list(
            map(lambda txt: count_by_lambda(lambda word: ed.is_emoticon(word) and ed.is_positive(word), txt), 
                   self.processed_Traindata["splitted_text"]))
        self.add_column("number_of_positive_emo", positive_emo)
        
        #number of negative emoticons
        negative_emo = list(
            map(lambda txt: count_by_lambda(lambda word: ed.is_emoticon(word) and not ed.is_positive(word), txt), 
                   self.processed_Traindata["splitted_text"]))
        self.add_column("number_of_negative_emo", negative_emo)
        
        #number of interjections
        interjections = list(map(lambda txt: count_interjections(txt),
                               self.processed_Traindata["splitted_text"]))
        self.add_column("number_of_interjections", interjections)

In [99]:
data = ExtraFeatures()
data.initialize("../data/LDAOutput.csv")
data.build_features()
data.cleaningData(DataPreprocessing())
data.tokenize()
data.stem()
data.buildWordlist()
data_model, labels = data.build_data_model()

data_model

,number_of_uppercase,number_of_exclamation,number_of_question,number_of_ellipsis,number_of_hashtags,number_of_mentions,number_of_quotes,number_of_urls,number_of_positive_emo,number_of_negative_emo,...,gorgeou_bow,woohoo_bow,thousand_bow,understat_bow,furiou_bow,manual_bow,smell_bow,ber_bow,charleston_bow,nrt_bow
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [100]:
#Load Naive-Bayes Library
with open('../model/NaiveBayesClassifier.pkl', 'rb') as nid:
    nb_loaded = cPickle.load(nid)
with open('../model/RandomForestClassifier.pkl', 'rb') as rid:
    rf_loaded = cPickle.load(rid)

result_nb = nb_loaded.predict(data_model)
print(type(result_nb))
print("Naive-Bayes Prediction : ",result_nb)
result_rf = rf_loaded.predict(data_model)
print("Random Forest Prediction : ",result_rf)
df_csv = pd.read_csv("../data/LDAOutput.csv")
df_csv['NaiveBayesSentiment'] = pd.DataFrame(result_nb)
df_csv['RandomForestSentiment'] = pd.DataFrame(result_rf)

df_csv.to_csv("../data/LDAOutput.csv")

<class 'numpy.ndarray'>
Naive-Bayes Prediction :  ['negative']
Random Forest Prediction :  ['negative']
